## CMIP6 sea ice concentration maps

## Import libraries

In [ ]:
import math
import warnings

import cartopy.crs as ccrs
import cmocean
import matplotlib.pyplot as plt
import xarray as xr
from c3s_eqc_automatic_quality_control import diagnostics, download, plot

plt.style.use("seaborn-v0_8-notebook")
warnings.filterwarnings("ignore", module="cf_xarray")

## Set parameters

In [ ]:
# Time Periods
periods = [
    slice("1985", "1994"),
    slice("1995", "2004"),
    slice("2005", "2014"),
]

# Regions
regions = ("northern_hemisphere", "southern_hemisphere")
assert set(regions) <= {"northern_hemisphere", "southern_hemisphere"}

# Choose CMIP6 historical models
models = [
    "access_cm2",
    "access_esm1_5",
    "cams_csm1_0",
    "canesm5",
    "canesm5_canoe",
    "cmcc_cm2_hr4",
    "cmcc_cm2_sr5",
    "cmcc_esm2",
    "cnrm_cm6_1",
    "cnrm_cm6_1_hr",
    "cnrm_esm2_1",
    "e3sm_1_0",
    "e3sm_1_1",
    "e3sm_1_1_eca",
    "ec_earth3_aerchem",
    "ec_earth3_cc",
    "ec_earth3_veg_lr",
    "fgoals_f3_l",
    "giss_e2_1_h",
    "hadgem3_gc31_ll",
    "hadgem3_gc31_mm",
    "inm_cm4_8",
    "inm_cm5_0",
    "ipsl_cm5a2_inca",
    "ipsl_cm6a_lr",
    "miroc6",
    "miroc_es2l",
    "mpi_esm1_2_hr",
    "mpi_esm1_2_lr",
    "mri_esm2_0",
    "nesm3",
    "norcpm1",
    "taiesm1",
    "ukesm1_0_ll",
]

## Define requests

In [ ]:
request_cmip6_historical = (
    "projections-cmip6",
    {
        "format": "zip",
        "temporal_resolution": "monthly",
        "experiment": "historical",
        "variable": "sea_ice_area_percentage_on_ocean_grid",
        "month": [f"{month:02d}" for month in range(1, 13)],
    },
)

request_satellite = (
    "satellite-sea-ice-concentration",
    {
        "cdr_type": "cdr",
        "variable": "all",
        "version": "v2",
        "origin": "eumetsat_osi_saf",
    },
)

## Functions to cache

In [ ]:
def interpolate_to_satellite_grid(obj, region, **regrid_kwargs):
    # Remove nan columns
    for dim in [dim for dim in obj.dims if "x" in dim or "lon" in dim]:
        for i in (0, -1):
            if obj.isel({dim: i}).isnull().all():
                obj = obj.drop_isel({dim: i})

    collection_id = "satellite-sea-ice-concentration"
    request = {
        "region": region,
        "version": "v2",
        "variable": "all",
        "format": "zip",
        "origin": "esa_cci",
        "cdr_type": "cdr",
        "year": "2002",
        "month": "06",
        "day": "01",
    }
    grid_out = download.download_and_transform(collection_id, request).drop_dims("time")
    return diagnostics.regrid(obj, grid_out, **regrid_kwargs)


def compute_interpolated_sic(ds, period, region=None, **regrid_kwargs):
    # Get sic
    sic = ds.cf["sea_ice_area_fraction"]
    sic = sic.assign_coords({coord: ds[coord] for coord in ("longitude", "latitude")})

    # Deal with missing values
    mask = (sic.notnull() & (sic != 0)).any(tuple(set(sic.dims) - {"time"}))
    sic = sic.where(mask.compute(), drop=True)
    sic = sic.groupby("time.month").mean(keep_attrs=True)

    # Period mean
    sic = sic.mean("month", keep_attrs=True)

    # Regrid
    if regrid_kwargs:
        sic = interpolate_to_satellite_grid(sic, region=region, **regrid_kwargs)

    # Units
    if sic.attrs.get("units", "") == "(0 - 1)":
        sic *= 100
    sic.attrs["units"] = "%"

    # Interpolation
    return sic.to_dataset(name="sic")

## Download and transform CMIP6

In [ ]:
cmip6_datasets = {}
for region in regions:
    period_datasets = []
    for period in periods:
        model_datasets = []
        for model in models:
            print(f"{model=} {region=} {period=}")
            collection_id, request = request_cmip6_historical
            year_start = math.floor(int(period.start) / 10) * 10
            year_stop = min(2015, math.ceil(int(period.stop) / 10) * 10)
            ds = download.download_and_transform(
                collection_id,
                request
                | {
                    "model": model,
                    "year": [str(year) for year in range(year_start, year_stop)],
                },
                transform_func=compute_interpolated_sic,
                transform_func_kwargs={
                    "period": period,
                    "region": region,
                    "method": "nearest_s2d",
                    "periodic": True,
                    "ignore_degenerate": True,
                },
                transform_chunks=False,
                chunks={"year": 10},
            )
            model_datasets.append(
                ds.expand_dims(model=[model], period=[f"{period.start}-{period.stop}"])
            )
        period_datasets.append(xr.concat(model_datasets, "model"))
    cmip6_datasets[region] = xr.concat(period_datasets, "period")

## Download and transform satellite data

In [ ]:
satellite_datasets = {}
for region in regions:
    period_datasets = []
    for period in periods:
        print(f"{region=} {period=}")
        collection_id, request = request_satellite
        request = request | {"region": region}
        requests = download.update_request_date(
            request,
            start=f"{period.start}-01",
            stop=f"{period.stop}-12",
            stringify_dates=True,
        )
        ds = download.download_and_transform(
            collection_id,
            requests,
            transform_func=compute_interpolated_sic,
            transform_func_kwargs={"period": period},
            transform_chunks=False,
            chunks={"year": 1},
            # Parameters to speed up IO
            concat_dim="time",
            combine="nested",
            data_vars="minimal",
            coords="minimal",
            compat="override",
        )
        period_datasets.append(ds.expand_dims(period=[f"{period.start}-{period.stop}"]))
    satellite_datasets[region] = xr.concat(period_datasets, "period")

## Plot maps

In [ ]:
map_slices = {
    "northern_hemisphere": {"xc": slice(50, -100), "yc": slice(50, -50)},
    "southern_hemisphere": {"xc": slice(50, -50), "yc": slice(50, -50)},
}

for region in regions:
    da_satellite = satellite_datasets[region]["sic"]
    da_cmip6 = cmip6_datasets[region]["sic"].mean("model", keep_attrs=True)
    da_cmip6 = da_cmip6.where(da_satellite.notnull())

    da_cmip6 = da_cmip6.expand_dims(product=["CMIP6"])
    da_satellite = da_satellite.expand_dims(
        product=[request_satellite[1]["origin"].replace("_", "-").upper()]
    )
    projection = ccrs.Stereographic(
        central_latitude=90 if region.startswith("northern") else -90
    )
    da = xr.concat([da_satellite, da_cmip6], "product").isel(map_slices[region])
    da.attrs["long_name"] = "Sea ice concentration"

    plot.projected_map(
        da, col="product", row="period", projection=projection, cmap=cmocean.cm.ice
    )
    plt.show()

    with xr.set_options(keep_attrs=True):
        da_bias = da.diff("product")
    da_bias.attrs["long_name"] = "Bias in " + da_bias.attrs["long_name"].lower()
    plot.projected_map(
        da_bias,
        row="period",
        projection=projection,
        robust=True,
        cmap=cmocean.cm.balance,
    )
    plt.show()